# IMagic

This script is used to color the black and white image using the pre-trained parameters

### Importing required libraries

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import os
import h5py

/glob/intel-python/versions/2018u2/intelpython3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Loading, resizing and reshaping the image 

Here the image which is to be colored is loaded from the directory using open-cv library and then resized in 64x64 pixels image, further it is cob=nverted into a single vector to for feeding it into the neural network.

In [76]:
dir='datac/dataa/train/'
image=np.array(cv2.imread(dir+os.listdir(dir)[700]))
image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
im=cv2.resize(image,(64,64))
X_train=cv2.cvtColor(im,cv2.COLOR_RGB2GRAY).reshape(1,-1)
X_train=X_train.T
X_train.shape

### Creating placeholder to pass training data

In [37]:
def create_placeholder(n_x):
    X=tf.placeholder(dtype=tf.float32,shape=[n_x,None],name='X')
    
    return X


### Fetching the pre-trained parameters

In [38]:
def fetch_parameters(param):
    
    #print(type(param['W1']))
    W1=param['W1'],
    b1=param['b1'],
    W2=param['W2'],
    b2=param['b2']
    
    parameters={
        "W1":W1,
        "b1":b1,
        "W2":W2,
        "b2":b2
    }
    
    return parameters


### Forward propagation

In [39]:
def forward(X,parameters):
    
    W1=parameters['W1']
    b1=parameters['b1']
    W2=parameters['W2']
    b2=parameters['b2']
   
    A1=tf.add(tf.matmul(W1,X),b1)
    A2=tf.add(tf.matmul(W2,A1),b2)
    
    return A2


### Predicting the results

In [55]:
#This function predicts the output sing tensorflow library
def predict(X_train,param=None):
    
    tf.reset_default_graph()
    (n_x,_)=X_train.shape
    
    #create palceholders 
    X=create_placeholder(n_x)
    
    para=fetch_parameters(param)
    
    A2=forward(X,para)
    
    #Initialize all variables
    init=tf.global_variables_initializer()
    
    #Start session
    with tf.Session() as session:
        session.run(init)
        result=session.run(A2,feed_dict={X:X_train})
        
    return result
    
    
#This function predicts the output without using tensorflow library
def predict_b(X_train,parameters=None):
    
    W1=parameters['W1']
    b1=parameters['b1']
    W2=parameters['W2']
    b2=parameters['b2']
    
    X=np.float32(X_train)
    
    A1=np.add(np.matmul(W1,X),b1)
    A2=np.add(np.matmul(W2,A1),b2)
    
    return A2


### Loading the pre-trained parameters

In [36]:
param={}
filename = 'params1.hdf5'
f = h5py.File(filename, 'r')
param['W1']=f['w1'][()]
param['b1']=f['b1'][()]
param['W2']=f['w2'][()]
param['b2']=f['b2'][()]
cost=f['cost'][()]
f.close()

### Predicting the results

In [71]:
pred=predict_b(X_train=X_train,parameters=param)

### Printing the output images

In [ ]:
output=pred.T

a=0
for var in range(1):
    image=output[var+a].reshape(64,64,3)
    image=np.uint8(image)
    image=cv2.resize(image,(256,256))
    plt.figure(var+1)
    plt.imshow(image)

plt.show()